# Seminar Project: Subsetting Speeches
---------------
``` 

Jakob Amann
Elias Gabriel Heppner, 1244414
Andri Rutschmann

### Loading Packages


In [1]:
import numpy as np 
import pandas as pd 
import time
import pickle
import time
import os
import datetime
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

# probably wise to switch to tiktoken as tokenizer as this is the one recommended by OpenAI 
import nltk
nltk.download('punkt')  # Nur notwendig beim ersten Mal
from nltk.tokenize import word_tokenize # import tokenizer to estimate token count of a speech


c:\Users\elias\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\elias\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Load Speeches Data


In [2]:
#Define 
os.chdir("../data/opendiscourse")
os.getcwd()

'c:\\Users\\elias\\Documents\\Master\\3. Semester\\LLM\\final_project\\Politricks\\data\\opendiscourse'

In [3]:

with open('speeches.pkl', 'rb') as f:
    # Load the object from the file
    speeches_df = pickle.load(f)

with open('politicians.pkl', 'rb') as f:
    # Load the object from the file
    pols_df = pickle.load(f)
    
with open('factions.pkl', 'rb') as f:
    # Load the object from the file
    facts_df = pickle.load(f)
    
with open('electoral_terms.pkl', 'rb') as f:
    # Load the object from the file
    elterms_df = pickle.load(f)
    
with open('contributions_extended.pkl', 'rb') as f:
    # Load the object from the file
   consex_df = pickle.load(f)
    
    

FileNotFoundError: [Errno 2] No such file or directory: 'speeches.pkl'

In [ ]:
speeches_df = pd.DataFrame(speeches_df)
pols_df = pd.DataFrame(pols_df)
facts_df = pd.DataFrame(facts_df)
elterms_df = pd.DataFrame(elterms_df)   
consex_df = pd.DataFrame(consex_df)

speeches_df

,id,session,electoral_term,politician_id,speech_content,faction_id,document_url,position_short,position_long,date
0,0,2,1,11001150,Meine Damen und Herren! Ich eröffne die 2. Sit...,-1,https://dip21.bundestag.de/dip21/btp/01/01002.pdf,Presidium of Parliament,präsident,1949-09-12
1,1,2,1,-1,"Der Bundesrat ist versammelt, Herr Präsident.\n",-1,https://dip21.bundestag.de/dip21/btp/01/01002.pdf,Guest,präsident des bundesrats,1949-09-12
2,2,2,1,11001150,Ich danke für diese Erklärung. Ich stelle dami...,-1,https://dip21.bundestag.de/dip21/btp/01/01002.pdf,Presidium of Parliament,präsident,1949-09-12
3,3,2,1,-1,"Ja, ich habe den Wunsch.\n",-1,https://dip21.bundestag.de/dip21/btp/01/01002.pdf,Guest,bundespräsident,1949-09-12
4,4,2,1,11001150,Ich erteile dem Herrn Bundespräsidenten das Wo...,-1,https://dip21.bundestag.de/dip21/btp/01/01002.pdf,Presidium of Parliament,präsident,1949-09-12
...,...,...,...,...,...,...,...,...,...,...
919518,1072832,38,20,11005267,\n\nSehr geehrte Frau Präsidentin! Liebe Kolle...,23,https://dip21.bundestag.de/dip21/btp/20/20038.pdf,Member of Parliament,NA,2022-05-20
919519,1072833,38,20,11003206,\n\nFür die CDU/CSU-Fraktion hat nun der Kolle...,-1,https://dip21.bundestag.de/dip21/btp/20/20038.pdf,Presidium of Parliament,Vizepräsidentin,2022-05-20
919520,1072834,38,20,11004384,\n\nSehr geehrte Frau Präsidentin! Liebe Kolle...,4,https://dip21.bundestag.de/dip21/btp/20/20038.pdf,Member of Parliament,NA,2022-05-20
919521,1072835,38,20,11003206,\n\nDas Wort hat die Kollegin Melanie Wegling ...,-1,https://dip21.bundestag.de/dip21/btp/20/20038.pdf,Presidium of Parliament,Vizepräsidentin,2022-05-20


In [ ]:
pols_df

,id,first_name,last_name,birth_place,birth_country,birth_date,death_date,gender,profession,aristocracy,academic_title
0,11000001,Manfred,Abelein,Stuttgart,Deutschland,1930-10-20,2008-01-17,männlich,"Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...",NA,Prof. Dr.
1,11000002,Ernst,Achenbach,Siegen,Deutschland,1909-04-09,1991-12-02,männlich,Rechtsanwalt und Notar,NA,Dr.
2,11000003,Annemarie,Ackermann,Parabutsch,Jugoslawien,1913-05-26,1994-02-18,weiblich,Hilfsreferentin,NA,NA
3,11000004,Else,Ackermann,Berlin,Deutschland,1933-11-06,2019-09-14,weiblich,Ärztin,NA,Dr.
4,11000005,Ulrich,Adam,Teterow,Deutschland,1950-06-09,NA,männlich,"Mathematiker, Geschäftsführer",NA,NA
...,...,...,...,...,...,...,...,...,...,...,...
4381,11005305,Bettina,Stark-Watzinger,NA,NA,NA,NA,NA,NA,NA,NA
4382,11005306,Manfred,Stolpe,NA,NA,NA,NA,NA,NA,NA,NA
4383,11005307,Johanna,Wanka,NA,NA,NA,NA,NA,NA,NA,NA
4384,11005308,Ludger,Westrick,NA,NA,NA,NA,NA,NA,NA,NA


In [ ]:
facts_df

,id,abbreviation,full_name
0,-1,not found,not found
1,0,AfD,Alternative für Deutschland
2,1,BHE,Block der Heimatvertriebenen und Entrechteten
3,2,BP,Bayernpartei
4,3,Grüne,Bündnis 90/Die Grünen
5,4,CDU/CSU,Christlich Demokratische Union Deutschlands/Ch...
6,5,DA,Demokratische Arbeitsgemeinschaft
7,6,DIE LINKE.,DIE LINKE.
8,7,DP,Deutsche Partei
9,8,DP/DBP,Deutsche Partei/Deutsche Partei Bayern


In [ ]:
elterms_df

,id,start_date,end_date
0,1,-641174400,-512524800
1,2,-512438400,-385516800
2,3,-385430400,-259113600
3,4,-259027200,-132710400
4,5,-132624000,-6393600
5,6,-6307200,92966400
6,7,93052800,219283200
7,8,219369600,342057600
8,9,342144000,417657600
9,10,417744000,540518400


In [ ]:
consex_df

,id,type,first_name,last_name,politician_id,content,speech_id,text_position,faction_id
0,0,Lachen,,,-1,links,6,7,-1
1,1,Personen-Einruf,,hütter,11000979,Nein!,6,2,13
2,2,Zuruf,,,-1,§ 51!,9,3,20
3,3,Zuruf,,,-1,links,11,0,-1
4,4,Personen-Einruf,,blücher,11000202,Jawohl!,12,0,13
...,...,...,...,...,...,...,...,...,...
2546207,2546207,Personen-Einruf,götz,frömming,11004722,Das gibt es gar nicht: gerechte Umverteilung!,1072836,2,0
2546208,2546208,Beifall,,,-1,,1072836,3,23
2546209,2546209,Beifall,,,-1,,1072836,3,13
2546210,2546210,Beifall,,,-1,,1072836,3,3


### Merge and Omit

In [ ]:
speeches_df = speeches_df[speeches_df["electoral_term"] == 18 or speeches_df["electoral_term"] == 19 speeches_df["electoral_term"] == 20]
speeches_df
fin_df = pd.merge(speeches_df, pols_df, left_on='politician_id', right_on='id', how='inner')
fin_df = pd.merge(fin_df, facts_df, left_on='faction_id', right_on='id', how='inner')
fin_df = fin_df[fin_df["electoral_term"] == 18 or fin_df["electoral_term"] == 19 fin_df["position_"] == 20]
fin_df = fin_df[["id_x", "session", "electoral_term", "date", "document_url", "speech_content", "politician_id", "first_name", "last_name", "faction_id",  "abbreviation", "position_short", "position_long"]]
fin_df
fin_df.to_pickle("sub_speeches.csv", index = False)